In [1]:
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets ,models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchsummary import summary


from __future__ import print_function, division

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

#  https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/62840b1eece760d5e42593187847261f/transfer_learning_tutorial.ipynb#scrollTo=dF0_RiFvCP4s


import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200


import os
import pandas as pd
from torchvision.io import read_image
#!pip install --upgrade gensim
from gensim.models import Word2Vec


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_features_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_features.csv'
test_features_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/test_features.csv'
train_labels_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_labels.csv'
test_labels_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_labels - Copy.csv'
def get_word2v (path):
  test = pd.read_csv(path)
  print(test.head())
  train = test['dna'].values.tolist()

  # print(len(train)) 121
  DNA_list = []
  for DNA in train:
    DNA = list(DNA)
    DNA_list.append(DNA)
  model = Word2Vec(DNA_list , vector_size = 10 , min_count= 1 )
  print(model , model.wv.key_to_index )
 
  
  vectorized = []
  for vec in DNA_list:
    v = np.array([])
    for char in vec :
      char = model.wv[char]
      v= np.append(v ,char)
    vectorized.append(v)

  return vectorized,model


# print(DNA_list)=['T', 'A', 'C', 'A', 'T', 'T', 'A', 'T', 'A', 'T', 'T', 'T', 'T', 'A', 'T', 'T', 'T', 'T', 'T', 'G', 'G', 'A', 'A', 'T', 'T', .......



In [4]:

def get_word2v_vectorsOnly (path,model):
  test = pd.read_csv(path)
  print(test.head())
  train = test['dna'].values.tolist()

  # print(len(train)) 121
  DNA_list = []
  for DNA in train:
    DNA = list(DNA)
    DNA_list.append(DNA)
  vectorized = []
  for vec in DNA_list:
    v = np.array([])
    for char in vec :
      char = model.wv[char]
      v= np.append(v ,char)
    vectorized.append(v)

  return vectorized

In [5]:

class CustomDNADataset(Dataset):
    def __init__(self, feature_matrix,label_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(label_file)
        self.img_features =feature_matrix
        self.transform = transform
        self.target_transform = target_transform


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
 ############# DNA transform ######################     
        DNA = self.img_features[idx]

        #print(len(DNA))

        n = np.array(DNA)# 10*dim*12090
        if len(DNA) < 6000 :

          Our_pad =6000-len(DNA)
          #print(len(DNA))
          
          for pad in range(Our_pad):# we pad the end of the sequence with zeros 
            n =np.append(n,[0,0,0,0,0,0,0,0,0,0])

        #print(n.shape)
        l = np.array(n , dtype=np.float32)
        #print(l.shape)
        Data_array = torch.from_numpy(l[0:6000])#crop first 800 features 3*22*22*10

        DNA_image = Data_array.reshape(3,40,50)


#################################################
        label = self.img_labels.iloc[idx, 1]
        # we get the labels from a different file 

        return DNA_image, label

In [6]:
# train_features_path = '/Users/yousi/Untitled Folder/dna-barcode-classification/data/train_features.csv'
# #test_features_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/test_features.csv'
# train_labels_path = '/Users/yousi/Untitled Folder/dna-barcode-classification/data/train_labels.csv'


vectorized , model_w2vec =  get_word2v(train_features_path)

full_dataset= CustomDNADataset(vectorized,label_file=train_labels_path)


   id                                                dna
0   1  AACATTATACTTTATTTTCGGAGCATGATCAGGAATAGTAGGAACT...
1   2  TACACTATACTTCATTTTTGGTGCTTGAGCAGGAATGCTAGGAACA...
2   3  ----------------------------------------------...
3   4  AACATTATATTTTATTTTTGGTGCATGAGCTGGAATAGTAGGAACT...
4   5  AACTTTATATTTTATTTTTGGAGCTTGAGCTGGAATAGTTGGAACA...
Word2Vec(vocab=12, vector_size=10, alpha=0.025) {'T': 0, 'A': 1, 'C': 2, 'G': 3, '-': 4, 'N': 5, 'Y': 6, 'W': 7, 'R': 8, 'M': 9, 'S': 10, 'K': 11}


In [7]:
# for i, (images, labels) in enumerate(full_dataset):
#   print(i ,images.shape , labels)

In [8]:
# test_features_path = '/Users/yousi/Untitled Folder/dna-barcode-classification/data/test_features.csv'
# test_labels_path = '/Users/yousi/Untitled Folder/dna-barcode-classification/data/train_labels - Copy.csv'


vectorized_test = get_word2v_vectorsOnly(test_features_path,model_w2vec)

#full_dataset= CustomDNADataset(feature_file=train_features_path,label_file=train_labels_path)
# so pass the whole set by specifing the path of file to be inputed to our custum class 

Test_dataset = CustomDNADataset(vectorized_test,label_file=test_labels_path)
# for the test_dataset I am using fake label data which is not accually there just so that I can use 
# my custom class , it has the same len as the train data ( I choped some in csv file )

   id                                                dna
0   1  AATACTTTATTTTATTTTTGCTATATGAGCTGGAATAATTGGAGCT...
1   2  GGAGTATGATCTGGAATAGTCGGAACTTCTCTAAGAATTTTAATTC...
2   3  TACTTTATACTTTATTTTTGGAGCATGATCAGGAATAGTAGGAACT...
3   4  GATATTATATTTTATTTTTGGAATATGATCAGGAATAGTAGGAAGA...
4   5  ----------------------------------------------...


In [9]:
train_size = int(0.95 * len(full_dataset))
# this is a creative way to create a validation set by spliting the dataset 
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])


In [10]:
dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=15,
                                             shuffle=True, num_workers=2),
               'val' : torch.utils.data.DataLoader(validation_dataset, batch_size=15,
                                             shuffle=True, num_workers=2)}
dataset_sizes = {'train': len(train_dataset) ,
                 'val' : len(validation_dataset)}





In [11]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 


# hyper parameters 
input_size = 36*36
in_channel = 3
num_classes = 1214
learning_rate = 0.001
batch_size = 15
# the batch size appered to be a significant factor on the effectivness of the training
# 15 is found to be a good num 
num_epochs = 3
#


In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [14]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [16]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
train Loss: 1.4506 Acc: 0.8379
val Loss: 1.0694 Acc: 0.8529

Epoch 1/19
----------
train Loss: 0.7976 Acc: 0.9090
val Loss: 0.7200 Acc: 0.9040

Epoch 2/19
----------
train Loss: 0.5034 Acc: 0.9400
val Loss: 0.5755 Acc: 0.9241

Epoch 3/19
----------
train Loss: 0.3446 Acc: 0.9563
val Loss: 0.4762 Acc: 0.9396

Epoch 4/19
----------
train Loss: 0.2351 Acc: 0.9698
val Loss: 0.4063 Acc: 0.9427

Epoch 5/19
----------
train Loss: 0.1669 Acc: 0.9764
val Loss: 0.4202 Acc: 0.9443

Epoch 6/19
----------
train Loss: 0.0964 Acc: 0.9889
val Loss: 0.3720 Acc: 0.9551

Epoch 7/19
----------
train Loss: 0.0763 Acc: 0.9929
val Loss: 0.3645 Acc: 0.9520

Epoch 8/19
----------
train Loss: 0.0672 Acc: 0.9949
val Loss: 0.3664 Acc: 0.9536

Epoch 9/19
----------
train Loss: 0.0626 Acc: 0.9958
val Loss: 0.3630 Acc: 0.9567

Epoch 10/19
----------
train Loss: 0.0599 Acc: 0.9961
val Loss: 0.3648 Acc: 0.9536

Epoch 11/19
----------
train Loss: 0.0555 Acc: 0.9968
val Loss: 0.3642 Acc: 0.9582

Ep

In [20]:
test_loader = DataLoader(Test_dataset  , batch_size=1 , shuffle=False)
def write_csv_results (loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()
    your_file = open('word2vec_transferLearningCNN_30.csv', 'ab')
    # we create a csv file or open it 'ab' for append binery 
    sm = torch.nn.Softmax()
    # we use a softmax to compute the probability of each guess of the network 
    
    

    with torch.no_grad():
      for x,_ in loader:
        x = x.to(device = device )
       
        
        scores = model(x)
        _,prediction = scores.max(1)
        #print(scores.max())
        #print('model prediction = ',prediction , 'y = ',y)
        probabilities = sm(scores) 
        #print(probabilities.max())
        #print(prediction)
        if probabilities.max() < 0.30 :
          prediction = torch.tensor([-1])
          np.savetxt(your_file,prediction.cpu())
          # we save the prediction in the open file but first we have to 
          # send it back to being a cpu 
        else:
          np.savetxt(your_file,prediction.cpu().numpy())  
        
        

      model.train()
      your_file.close()

write_csv_results(test_loader,model_ft)
# using our def 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
